# The deconfounder: a PCA factor model + a logistic outcome model

In [1]:
import numpy.random as npr
import statsmodels.api as sm 
import scipy 
import numpy as np

from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.datasets import make_spd_matrix
from scipy import stats

stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)


/Users/yixinwang/py3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# import time
# timenowseed = int(time.time())
# npr.seed(timenowseed)
# print(timenowseed)
npr.seed(1534727263)

In [3]:
n = 10000 # number of data points
d = 3 # number of causes (=2) + number of confounders (=1)

# A simulated dataset

## simulate correlated causes

In [4]:
corrcoef = 0.4
stdev = np.ones(d)
corr = np.eye(d) * (1-corrcoef) + np.ones([d,d]) * corrcoef
print("correlation \n", corr)
b = np.matmul(stdev[:,np.newaxis], stdev[:,np.newaxis].T)
cov = np.multiply(b, corr)
mean = np.zeros(d)
# cov = make_spd_matrix(3)
print("covariance \n", cov)
X = npr.multivariate_normal(mean, cov, n)

correlation 
 [[1.  0.4 0.4]
 [0.4 1.  0.4]
 [0.4 0.4 1. ]]
covariance 
 [[1.  0.4 0.4]
 [0.4 1.  0.4]
 [0.4 0.4 1. ]]


## simulate the outcome

In [5]:
coef = np.array([0.2, 1.0, 0.9])
assert len(coef) == d
intcpt = 0.4

In [6]:
y = npr.binomial(1, np.exp(intcpt+coef.dot(X.T))/(1+np.exp(intcpt+coef.dot(X.T))))

# noncausal estimation: classical logistic regression

In [7]:
obs_n = d - 1

In [8]:
obs_X = X[:,:obs_n]

In [9]:
#ignore confounder
x2 = sm.add_constant(obs_X)
models = sm.Logit(y,x2)
result = models.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.540806
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Thu, 20 Sep 2018   Pseudo R-squ.:                  0.2107
Time:                        01:54:58   Log-Likelihood:                -5408.1
converged:                       True   LL-Null:                       -6851.6
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3640      0.024     15.325      0.000       0.317       0.411
x1             0.4353      0.

*   The true causal coefficient is (0.2, 1.0). 
*   But with the classical logistic regression, none of the confidence intervals include the truth.

# causal inference: the deconfounder with a PCA factor model

## fit a PCA

In [10]:
n_comp = 1
eps = 0.1

In [11]:
pca = PCA(n_components=n_comp)
pca.fit(obs_X)

PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [12]:
pca.components_

array([[-0.7122818 , -0.70189361]])

In [13]:
print(pca.explained_variance_ratio_)  

[0.70374746]


## compute the substitute confounder Z and the reconstructed causes A

In [14]:
Z = obs_X.dot(pca.components_.T) + npr.normal(scale=eps,size=(n,1))

In [15]:
A = np.dot(pca.transform(obs_X)[:,:n_comp], pca.components_[:n_comp,:]) + npr.normal(scale=eps,size=(n,obs_n))

In [16]:
X_pca_A = np.hstack((obs_X, A))
X_pca_Z = np.hstack((obs_X, Z))

## causal estimation with the reconstructed causes A

In [17]:
x2 = sm.add_constant(X_pca_A)
models = sm.Logit(y,x2)
result = models.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.540465
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9995
Method:                           MLE   Df Model:                            4
Date:                Thu, 20 Sep 2018   Pseudo R-squ.:                  0.2112
Time:                        01:54:58   Log-Likelihood:                -5404.7
converged:                       True   LL-Null:                       -6851.6
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3572      0.024     14.809      0.000       0.310       0.404
x1             0.1832      0.

*   The true causal coefficient is (0.2, 1.0). 
*   But with the deconfounder, both of the confidence intervals (for x1, x2) include the truth.

## causal estimation with the substitute confounder Z

In [18]:
x2 = sm.add_constant(X_pca_Z)
models = sm.Logit(y,x2)
result = models.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.540708
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Thu, 20 Sep 2018   Pseudo R-squ.:                  0.2108
Time:                        01:54:58   Log-Likelihood:                -5407.1
converged:                       True   LL-Null:                       -6851.6
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3638      0.024     15.314      0.000       0.317       0.410
x1             0.2002      0.

*   The true causal coefficient is (0.2, 1.0). 
*   But with the deconfounder, both of the confidence intervals (for x1, x2) include the truth.

# The oracle case: when the confounder is observed

In [19]:
# oracle
x2 = sm.add_constant(X)
models = sm.Logit(y,x2)
result = models.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.496111
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Thu, 20 Sep 2018   Pseudo R-squ.:                  0.2759
Time:                        01:54:58   Log-Likelihood:                -4961.1
converged:                       True   LL-Null:                       -6851.6
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4149      0.025     16.480      0.000       0.366       0.464
x1             0.2405      0.

*   The true causal coefficient is (0.2, 1.0). 
*   When the confounder is observed, both of the confidence intervals (for x1, x2) include the truth.
*   The estimate is (expectedly) more efficient than the deconfounder.